<a href="https://colab.research.google.com/github/moneletizi94/face-mask-detector/blob/main/FaceMaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
INIT_LR = 1e-4
EPOCHS = 20
BATCH_SIZE = 32

In [ ]:
#add kaggle.json file in the machine (Kaggle->Profile->Create new API Token)
! pip install -q kaggle
from google.colab import files
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d sumansid/facemask-dataset
! unzip facemask-dataset.zip
! kaggle datasets download -d dhruvmak/face-mask-detection
! unzip face-mask-detection.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
facemask-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  facemask-dataset.zip
replace Mask/Mask/Mask109.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! mv Mask/Mask/* merged_dataset/with_mask/
! mv No\ Mask/No\ Mask/* merged_dataset/without_mask/

In [ ]:
! mv dataset/with_mask/* merged_dataset/with_mask/
! mv dataset/without_mask/* merged_dataset/without_mask/

In [ ]:
data = []
labels = []

In [ ]:
paths = ["merged_dataset/with_mask","merged_dataset/without_mask"]
for class_ in paths:
  for img in os.listdir(class_):
    img_path = os.path.join(class_,img)
    image = load_img(img_path, target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(class_)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
#transform class in one-hot encoding vectors
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size = 0.2, stratify= labels, random_state= 42)

In [ ]:
aug = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.15, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True
)

In [ ]:
base_model= MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
model = tf.keras.Sequential()
model.add(base_model)
model.add(AveragePooling2D(pool_size=(7,7)))
model.add(Flatten(name="flatten"))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))

In [ ]:
print('Compiling model...')
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
print('Model compiled')

Compiling model...
Model compiled


In [ ]:
history = model.fit(
    aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    steps_per_epoch = len(trainX) // BATCH_SIZE,
    validation_data = (testX, testY),
    validation_steps = len(testX) // BATCH_SIZE,
    epochs = EPOCHS
)

Epoch 1/20
19/19 [==============================] - 34s 2s/step - loss: 0.7507 - accuracy: 0.5747 - val_loss: 0.5280 - val_accuracy: 0.7692
Epoch 2/20
19/19 [==============================] - 31s 2s/step - loss: 0.5432 - accuracy: 0.7986 - val_loss: 0.4040 - val_accuracy: 0.9103
Epoch 3/20
19/19 [==============================] - 31s 2s/step - loss: 0.4565 - accuracy: 0.8552 - val_loss: 0.3274 - val_accuracy: 0.9295
Epoch 4/20
19/19 [==============================] - 31s 2s/step - loss: 0.3515 - accuracy: 0.9266 - val_loss: 0.2663 - val_accuracy: 0.9487
Epoch 5/20
19/19 [==============================] - 31s 2s/step - loss: 0.3036 - accuracy: 0.9365 - val_loss: 0.2210 - val_accuracy: 0.9551
Epoch 6/20
19/19 [==============================] - 31s 2s/step - loss: 0.2477 - accuracy: 0.9523 - val_loss: 0.1903 - val_accuracy: 0.9551
Epoch 7/20
19/19 [==============================] - 31s 2s/step - loss: 0.2233 - accuracy: 0.9466 - val_loss: 0.1676 - val_accuracy: 0.9615
Epoch 8/20
19/19 [==

In [ ]:
! pip install tensorflowjs
import tensorflowjs as tfjs
#save in tensorflow format
tfjs.converters.save_keras_model(model, 'tfjs_model')

     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 112kB 11.0MB/s 
  Found existing installation: tensorflow-hub 0.12.0
    Uninstalling tensorflow-hub-0.12.0:
      Successfully uninstalled tensorflow-hub-0.12.0


In [ ]:
!zip -r merged_dataset.zip merged_dataset/

  adding: merged_dataset/ (stored 0%)
  adding: merged_dataset/.ipynb_checkpoints/ (stored 0%)
  adding: merged_dataset/with_mask/ (stored 0%)
  adding: merged_dataset/with_mask/Mask66.jpg (deflated 2%)
  adding: merged_dataset/with_mask/Mask180.jpg (deflated 2%)
  adding: merged_dataset/with_mask/Mask289.jpg (deflated 1%)
  adding: merged_dataset/with_mask/image_165.png (deflated 0%)
  adding: merged_dataset/with_mask/image_118.png (deflated 0%)
  adding: merged_dataset/with_mask/image_191.png (deflated 0%)
  adding: merged_dataset/with_mask/Mask30.jpg (deflated 0%)
  adding: merged_dataset/with_mask/image_157.png (deflated 0%)
  adding: merged_dataset/with_mask/image_29.png (deflated 0%)
  adding: merged_dataset/with_mask/Mask73.jpeg (deflated 0%)
  adding: merged_dataset/with_mask/image_7.png (deflated 0%)
  adding: merged_dataset/with_mask/image_190.png (deflated 0%)
  adding: merged_dataset/with_mask/Mask5.jpg (deflated 1%)
  adding: merged_dataset/with_mask/Mask112.jpeg (deflated

In [ ]:
model.save('my_model')

In [ ]:
image = load_img('pedro-nomask.PNG', target_size=(224,224))
image = img_to_array(image)
#image = preprocess_input(image)
print(model.predict(np.array([image]), batch_size=1))

image = load_img('pedro-withmask.PNG', target_size=(224,224))
image = img_to_array(image)
#image = preprocess_input(image)
print(model.predict(np.array([image]), batch_size=1))


[[0.01511231 0.9848877 ]]
[[0.02196917 0.97803086]]
